In [1]:
from PIL import Image
import numpy as np
import os
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
imgdata = np.load( "E:/0212/photo/299_train_total.npy" ) 

In [3]:
imgdata.shape

(23241, 299, 299)

In [4]:
x_downTrain4D = np.expand_dims(imgdata, axis=3)
print(x_downTrain4D.shape)

(23241, 299, 299, 1)


In [5]:
labs_x = []
label_count = {}

for i in range(10):
    path = "E:/0212/photo/{}f/".format(i)
    file_count = len([name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))])
    label_count[str(i)] = file_count

for key, label_count in label_count.items():
    key = int(key)
    for i in range(label_count):
        labs_x.append(key)

print(len(labs_x))

23241


In [6]:
labs_xOneHot = np_utils.to_categorical(labs_x)

In [7]:
labs_xOneHot.shape

(23241, 10)

In [8]:
import resnet
img_rows, img_cols = 299, 299
img_channels = 1
mixmodel = resnet.ResnetBuilder.build_resnet_34((img_channels, img_rows, img_cols), 10)

# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# mixmodel = InceptionResNetV2(weights=None, classes=10, pooling="max", input_shape=(299, 299, 1))
mixmodel.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
print(mixmodel.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 150, 150, 64) 3200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 150, 150, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 150, 150, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [10]:
x_downTrain4D.shape

(23241, 299, 299, 1)

In [11]:
x_train_history = mixmodel.fit(x=x_downTrain4D,
                               y=labs_xOneHot, validation_split=0,
                               epochs=40, batch_size=40, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/40
23241/23241 [==============================] - 152s 7ms/step - loss: 1.1821 - categorical_accuracy: 0.9250
Epoch 2/40
23241/23241 [==============================] - 141s 6ms/step - loss: 0.4520 - categorical_accuracy: 0.9903
Epoch 3/40
23241/23241 [==============================] - 141s 6ms/step - loss: 0.3334 - categorical_accuracy: 0.9922
Epoch 4/40
23241/23241 [==============================] - 141s 6ms/step - loss: 0.2512 - categorical_accuracy: 0.9939
Epoch 5/40
23241/23241 [==============================] - 139s 6ms/step - loss: 0.2341 - categorical_accuracy: 0.9940
Epoch 6/40
23241/23241 [==============================] - 139s 6ms/step - loss: 0.2119 - categorical_accuracy: 0.9941
Epoch 7/40
23241/23241 [==============================] - 139s 6ms/step - loss: 0.1974 - categorical_accuracy: 0.9935
Epoch 8/40
23241/23241 [==============================] - 139s 6ms/step - loss: 0.1629 - categorical_accuracy: 0.9952
Epoch 9/

In [12]:
file_path = "E:/model_299_rsnet_34"

with open("{}.json".format(file_path), "w") as f:
    f.write(mixmodel.to_json())

mixmodel.save_weights("{}.h5".format(file_path))